# A comparison of dense and sparse matrix vector multiplication

In [1]:
import numpy as np
import scipy.sparse

## Setup

In [2]:
p = 0.01
Nc, Na = 10000, 1000
c = np.ones(Nc)
a = np.ones(Na)
K = np.random.random((Nc, Na)) < p

## Dense matrix vector multiplication

In [3]:
%timeit np.dot(K, a)

100 loops, best of 3: 20.8 ms per loop


In [4]:
%timeit np.dot(c, K)

10 loops, best of 3: 26.6 ms per loop


## Sparse matrix vector multiplication

In [5]:
Ksp = scipy.sparse.csr_matrix(K)

In [6]:
%timeit scipy.sparse.csr_matrix(K)

10 loops, best of 3: 33.5 ms per loop


In [7]:
np.all(Ksp.dot(a) == K.dot(a))

True

In [8]:
%timeit Ksp.dot(a)

1000 loops, best of 3: 258 µs per loop


In [9]:
np.all(Ksp.T.dot(c) == np.dot(c, K))

True

In [10]:
%timeit Ksp.T.dot(c)

1000 loops, best of 3: 301 µs per loop


## Conclusions

Using sparse matrices can provide huge speed-ups even for moderate sparsity (here 1% non-zero entries) in high dimensions.